# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
total_schools = school_data_complete['School ID'].nunique()
total_students = school_data_complete['Student ID'].nunique()

budget_by_schoolID = school_data_complete.groupby(['budget'], as_index=False).mean()
total_budget = budget_by_schoolID['budget'].sum()
 

avg_math_sc = school_data_complete['math_score'].mean()
avg_reading_sc = school_data_complete['reading_score'].mean()
school_data_complete.head()

overall_passing_rate = (avg_math_sc + avg_reading_sc)/2

passing_math_count = school_data_complete.loc[school_data_complete['math_score'] >= 70]
total_passing_math_st = passing_math_count['Student ID'].nunique()
perc_passing_math = (total_passing_math_st/total_students)*100

passing_reading_count = school_data_complete.loc[school_data_complete['reading_score'] >= 70]
total_passing_reading_st = passing_reading_count['Student ID'].nunique()
perc_passing_reading = (total_passing_reading_st/total_students)*100

summary_df = pd.DataFrame({
    'Total Schools': total_schools,
    'Total Students': total_students,
    'Total Budget': total_budget,
    'Average Math Score': avg_math_sc,
    'Average Reading Score': avg_reading_sc,
    '% Passing Marth': perc_passing_math,
    '% Passing Reading': perc_passing_reading,
    '% Overall Passing Rate': overall_passing_rate
}, index = [0])
summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Marth,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.431606


In [3]:
school_summary = school_data_complete.groupby(['School ID','type','school_name'], as_index=False).mean()

school_summary.rename(columns = {
    'size':'Total Students',
    'budget':'Total School Budget',
    'reading_score':'Average Reading Score',
    'math_score':'Average Math Score'
},inplace=True)

school_summary['Per Student Budget'] = school_summary['Total School Budget']/school_summary['Total Students']
school_summary.drop(columns=['Student ID'], inplace=True)

passing_scores = pd.DataFrame({
    'School Name': '',
    'Passing Math': '',
    'Passing Reading':''}, index=[0])
passing_scores

ind = 1

list_of_schools = school_summary['school_name'].unique().tolist()

for school in list_of_schools:
    school_math_p = len(school_data_complete[(school_data_complete['school_name'] == school) & (school_data_complete['math_score'] >= 70)])
    school_reading_p = len(school_data_complete[(school_data_complete['school_name'] == school) & (school_data_complete['reading_score'] >= 70)])
    current_school = pd.DataFrame({
    'School Name': school,
    'Passing Math': school_math_p,
    'Passing Reading': school_reading_p}, 
    index=[ind])
    ind = ind+1
    passing_scores = passing_scores.append(current_school)
    

school_data_complete2 = pd.merge(school_summary, passing_scores, how="left", left_on=['school_name'], right_on=['School Name'])
school_data_complete2['% Passing Math'] = (school_data_complete2['Passing Math']/school_data_complete2['Total Students'])*100
school_data_complete2['% Passing Reading'] = (school_data_complete2['Passing Reading']/school_data_complete2['Total Students'])*100
school_data_complete2['% Overall Passing Rate'] = (school_data_complete2['% Passing Math']+school_data_complete2['% Passing Reading'])/2
school_data_final = school_data_complete2.set_index('School Name')

school_data_final = school_data_final[['type','Total Students','Total School Budget','Per Student Budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']]
school_data_final

,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.6839,81.3164,73.5002
Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.9885,80.7392,73.3639
Shelton High School,Charter,1761.0,1056600.0,600.0,83.359455,83.725724,93.8671,95.8546,94.8609
Hernandez High School,District,4635.0,3022020.0,652.0,77.289752,80.934412,66.753,80.863,73.808
Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.3924,97.139,95.2657
Wilson High School,Charter,2283.0,1319574.0,578.0,83.274201,83.989488,93.8677,96.5396,95.2037
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.1335,97.0398,95.5867
Bailey High School,District,4976.0,3124928.0,628.0,77.048432,81.033963,66.6801,81.9333,74.3067
Holden High School,Charter,427.0,248087.0,581.0,83.803279,83.814988,92.5059,96.2529,94.3794


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [4]:
school_data_final.sort_values(by=['% Overall Passing Rate'], ascending=False, inplace=True)
school_data_final.head()

,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.1335,97.0398,95.5867
Thomas High School,Charter,1635.0,1043130.0,638.0,83.418349,83.848930,93.2722,97.3089,95.2905
Pena High School,Charter,962.0,585858.0,609.0,83.839917,84.044699,94.5946,95.9459,95.2703
Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.3924,97.139,95.2657
Wilson High School,Charter,2283.0,1319574.0,578.0,83.274201,83.989488,93.8677,96.5396,95.2037


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [5]:
school_data_final.sort_values(by=['% Overall Passing Rate'], inplace=True)
school_data_final.head()

,type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999.0,2547363.0,637.0,76.842711,80.744686,66.3666,80.2201,73.2933
Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.9885,80.7392,73.3639
Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.6839,81.3164,73.5002
Johnson High School,District,4761.0,3094650.0,650.0,77.072464,80.966394,66.0576,81.2224,73.64
Ford High School,District,2739.0,1763916.0,644.0,77.102592,80.746258,68.3096,79.299,73.8043


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
grouped_data = school_data_complete.groupby(['school_name','grade'], as_index=False).mean()
grouped_data['grade'] = pd.Categorical(grouped_data['grade'],['9th','10th','11th','12th'])
grouped_data.sort_values(by=['grade'], inplace=True)
grouped_data.pivot(index='school_name',columns='grade', values='math_score')

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
grouped_data = school_data_complete.groupby(['school_name','grade'], as_index=False).mean()
grouped_data['grade'] = pd.Categorical(grouped_data['grade'],['9th','10th','11th','12th'])
grouped_data.sort_values(by=['grade'], inplace=True)
grouped_data.pivot(index='school_name',columns='grade', values='reading_score')

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [18]:
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
school_data_final['Spending Ranges (Per Student)'] = pd.cut(school_data_final['Per Student Budget'], spending_bins, labels=group_names)
grouped_df = school_data_final.groupby('Spending Ranges (Per Student)')
grouped_df[['Average Math Score','Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']].mean()

,Average Math Score,Average Reading Score
Spending Ranges (Per Student),,
<$585,83.455399,83.933814
$585-615,83.599686,83.885211
$615-645,79.079225,81.891436
$645-675,76.997210,81.027843


## Scores by School Size

* Perform the same operations as above, based on school size.

In [17]:
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_data_final['School Size'] = pd.cut(school_data_final['Total Students'], size_bins, labels=group_names)
grouped_df = school_data_final.groupby('School Size')
grouped_df[['Average Math Score','Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']].mean()

,Average Math Score,Average Reading Score
School Size,,
Small (<1000),83.821598,83.929843
Medium (1000-2000),83.374684,83.864438
Large (2000-5000),77.746417,81.344493


## Scores by School Type

* Perform the same operations as above, based on school type.

In [16]:
grouped_df = school_data_final.groupby('type')
grouped_df[['Average Math Score','Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']].mean()

,Average Math Score,Average Reading Score
type,,
Charter,83.473852,83.896421
District,76.956733,80.966636
